# 2 st Assignment: Mixed-Integer Linear Programming

Generate random data, consider n=500 observations (i=1,…,500), from a predefined linear regression model with m=20 variables (j=1,…,20). Assume that the regression coefficients are integers so that −5 ≤ 𝛽 𝑗 ≤ 5. Assume also independent normal residuals.

$$ Y = \beta ' X + \epsilon $$

In [3]:
# Imports

import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

## Exercise 1

The classical least squares approach is to find the values of vector 𝛽 = (𝛽 1 , … , 𝛽 𝑗 , … , 𝛽 𝑚 ) 𝑇 so that:

$$ \min_{\beta} \sum^n_{i=1} (y_i - \beta ' x_i)^2) $$
    
where $x_i = (x_{1i} , … , x_{2i} , … , x_{mi} )$ 𝑇 for $ i = 1, ...,n.$

Estimate the value of the regression coefficients by using the analytical solution for the least squares estimation problem. Tip:

$$\beta_{ls} = (X^T X)^{-1}X^TY $$

In [49]:
# Create random dataset 
i,j = 500,20
reg = make_regression(i,j)
Y = reg[1].reshape(i,1)
X = np.column_stack((np.ones(i),np.asmatrix(reg[0])))

In [50]:
beta = np.linalg.inv(np.transpose(X)*X)*np.transpose(X)*Y
beta

matrix([[ -4.88498131e-15],
        [  2.22044605e-15],
        [  6.51655011e+00],
        [ -5.10702591e-15],
        [  1.90630116e+01],
        [  7.81599133e+01],
        [ -5.55111512e-15],
        [  4.00513113e+01],
        [  7.24928285e+01],
        [  7.89537469e+01],
        [ -9.32587341e-15],
        [ -1.77635684e-15],
        [ -5.77315973e-15],
        [  3.77475828e-15],
        [ -4.88498131e-15],
        [  9.93201764e+01],
        [  1.77635684e-15],
        [  4.83800014e+01],
        [  2.12171520e+01],
        [  5.10702591e-15],
        [  7.15557054e+01]])

## Exercise 2

As an alternative, the least absolute value approach seeks to find the values of β by solving the following problem:

$$ \min_{\beta} \sum^n_{i=1} |y_i - \beta ' x_i| $$

Propose and implement in Pyomo an equivalent linear formulation for this problem. Compare the resulting β coefficients with the ones obtained in exercise 1).

## Exercise 3

Now assume that we want to impose the condition that only k factors (variables) affect the dependent variable Y. Extend the formulation in exercise 2 to a MILP (mixed integer linear optimization problem) to model the additional condition that up to k out of the β j coefficients have nonzero values.

## Exercise 4

Solve the problem in exercise 3 in Pyomo for k=1,…,20 and represent the behavior of the objective function with respect to k.

## Exercise 5

Another possibility to find β is the robust linear regression problem that exploits the robustness of the error median against outliers. It can be formulated as:

$$ \min_{\beta} \, median\, (|y_1-\beta ' x_1\, ..., |y_n - \beta ' x_n| ) $$

Formulate this model as a MILP and implement and solve it in Pyomo. Due to its computational complexity, consider as an input n=30 random observations (i=1,...,30), from a linear regression model with m=5 variables (j=1,...,5).